# Ground-Cover Classification Activity

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
from osgeo import gdal

/anaconda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/anaconda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## Data

We're going to be looking at satellite images. Let's have a look at one.

In [4]:
!ls /data/rs/

2004.npy  2004_12class.npy  2010.npy  2010_12class.npy


In [6]:
import numpy as np

In [8]:
# satellite_2004_full = gdal.Open('/data/rs/2004.img').ReadAsArray() / 255
# satellite_2010_full = gdal.Open('/data/rs/2010.img').ReadAsArray() / 255
satellite_2004_full = np.load('/data/rs/2004.npy')
satellite_2010_full = np.load('/data/rs/2010.npy')

In [9]:
scale_by = .25
import scipy.misc
def scale_image(img, size):
    return np.array([scipy.misc.imresize(chan, size) for chan in img])
satellite_2004 = scale_image(satellite_2004_full, scale_by)
satellite_2010 = scale_image(satellite_2010_full, scale_by) 

In [10]:
rows, cols = satellite_2010.shape[1:]
assert rows, cols == satellite_2004.shape[1:]

In [13]:
def show_image(img, ax=None):
    if len(img.shape) == 3:
        img = np.transpose(img, (1,2,0))
    if ax is None:
        plt.imshow(img, interpolation='nearest')
    else:
        ax.imshow(img, interpolation='nearest')

def show_rgb(red, green, blue, ax=None):
    show_image(np.array([red, green, blue]), ax=ax)


Cambia el codigo para mostrar el imagen en la manera mejor que encontraste en la actividad pasada.

In [14]:
fig, axes = plt.subplots()
green, red, NIR, MIR = satellite_2010
show_rgb(red=MIR, green=NIR, blue=green)
#show_rgb(red=red, green=green, blue=green)

<IPython.core.display.Javascript object>

# Define the Problem

We're going to try to identify ground cover.

* Input: a satellite image (12 channels)
* Output: which class we think each pixel is in. Specifically: is this pixel vegetation or not?

What makes a *good* output?

In [126]:
classes_2004 = scipy.misc.imresize(gdal.Open("../img/SPOT/2004_12class.img").ReadAsArray(), scale_by)
classes_2010 = scipy.misc.imresize(gdal.Open("../img/SPOT/2010_12class.img").ReadAsArray(), scale_by)

In [143]:
import sklearn.cluster
cluster = sklearn.cluster.MiniBatchKMeans(n_clusters=36)
np.reshape(satellite_2010, (4, -1)).shape
clustered_2010 = cluster.fit_predict(np.reshape(satellite_2010, (4, -1)).T).reshape(satellite_2010.shape[1:])

/Users/kcarnold/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:1279: DeprecationWarning: This function is deprecated. Please call randint(0, 3180565 + 1) instead
  0, n_samples - 1, init_size)
/Users/kcarnold/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:630: DeprecationWarning: This function is deprecated. Please call randint(0, 3180565 + 1) instead
  0, n_samples - 1, init_size)
/Users/kcarnold/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:630: DeprecationWarning: This function is deprecated. Please call randint(0, 3180565 + 1) instead
  0, n_samples - 1, init_size)
/Users/kcarnold/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:630: DeprecationWarning: This function is deprecated. Please call randint(0, 3180565 + 1) instead
  0, n_samples - 1, init_size)
/Users/kcarnold/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:1328: DeprecationWarning: This function is deprecated. Please call randin

In [139]:
def show_img_and_classes(red, green, blue, classes, true_classes):
    fig, axs = plt.subplots(1,3, sharex=True, sharey=True, figsize=(13,5))
    show_rgb(red=red, green=green, blue=blue, ax=axs[0])
    axs[1].imshow(classes)
    axs[2].imshow(classes == true_classes)

In [144]:
# Class 0: unknown
# Class 1: stream
true_classes_one_hot = np.zeros((2, rows, cols), dtype=bool)
true_classes_one_hot[1] = clustered_2010 == true_stream_idx
true_classes_one_hot[0] = true_classes_one_hot[1:].sum(axis=0) == 0
true_classes = np.argmax(true_classes_one_hot, axis=0)

In [147]:
def rectify(x): return np.maximum(x, 0)

In [152]:
class_scores = np.ones((2, rows, cols))
class_scores[1] = 0
# class_scores[1] = rectify(2*green + 1.1*red + .5*MIR - 250)
guessed_classes = np.argmax(class_scores, axis=0)
show_img_and_classes(red=MIR, green=NIR, blue=green, classes=guessed_classes, true_classes=true_classes)
print("Accuracy:", np.sum(guessed_classes == true_classes) / (rows * cols))

<IPython.core.display.Javascript object>

Accuracy: 0.99691344245


Your turn: try to get a higher accuracy.